<a href="https://colab.research.google.com/github/RybakovAlex/Python-for-Analythics/blob/main/%D0%91%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82_%22GB_Homework_3_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 3. Работа с библиотекой Pandas, часть 2. Манипуляции с данными


1. Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей интернет-магазина по полу.

In [250]:
# Импортируем нужную бибилиотеку
import pandas as pd

In [251]:
# Загружаем в Colab все датасеты
user_table = pd.read_csv('/content/1.user_table.csv')
user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [252]:
user_table.shape

(90400, 4)

In [253]:
home_page = pd.read_csv('/content/2.home_page_table.csv')
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [254]:
home_page.shape

(90400, 2)

In [255]:
search_page = pd.read_csv('/content/3.search_page_table.csv')
search_page.head()

,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page
3,780347,search_page
4,383739,search_page


In [256]:
search_page.shape

(45200, 2)

In [257]:
payment_page = pd.read_csv('/content/4.payment_page_table.csv')
payment_page.head()

,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page
3,901286,payment_page
4,195052,payment_page


In [258]:
payment_page.shape

(6030, 2)

In [259]:
payment_confirmation = pd.read_csv('/content/5.payment_confirmation_table.csv')
payment_confirmation.head()

,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page
3,538348,payment_confirmation_page
4,841681,payment_confirmation_page


In [260]:
payment_confirmation.shape

(452, 2)

In [261]:
# Меняем название одинаковой колонки в каждом датасете"
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [262]:
# Склеиваем последовательно все датасеты с помощью метода merge по идентификатору user_id"
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [263]:
df.shape

(90400, 8)

In [264]:
# Для удоства последующих расчетов, перекодируем названия событий в 0/1 с помощью лямбда-функции
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [265]:
# Импортируем графическую библиотеку Plotly (https://plotly.com/python/funnel-charts/)
from plotly import graph_objects as go

In [266]:
dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('sex').unstack('sex').reset_index()
# Уберем ненужный заголовок верхнего уровня
dfg.columns.name = None
# Переименуем стандартные после процедуры заголовки
dfg = dfg.rename(columns = {'index':'Page'})
dfg.head()

,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [267]:
fig = go.Figure(go.Funnel(
  y = dfg['Page'],
  x = dfg['Female'],
  textinfo = 'value + percent previous',
  name = 'Female',
  constraintext = 'outside',
  textposition = 'inside'
))

fig.add_trace(go.Funnel(
  y = dfg['Page'],
  x = dfg['Male'],
  textinfo = 'value + percent previous',
  name = 'Male',
  constraintext = 'outside',
  textposition = 'inside'
))

fig.update_layout(
  title = 'Воронка конверсии по полу пользователей',
  title_x=0.5,
  autosize=False,
  width=700,
  height=350
)
fig.show()

2. Создать 2 новые фичи на основе колонки “date”: месяц и день недели (пример https://stackoverflow.com/a/62624729, https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.weekday.html).

In [268]:
df['month'] = pd.DatetimeIndex(df['date']).month_name() # добавление колонки "month" (месяц)
df['day_of_week'] = pd.DatetimeIndex(df['date']).day_name() # добавление колонки "day_of_week" (день недели)
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,day_of_week
0,450007,2015-02-28,Desktop,Female,1,0,0,0,February,Saturday
1,756838,2015-01-13,Desktop,Male,1,0,0,0,January,Tuesday
2,568983,2015-04-09,Desktop,Male,1,1,0,0,April,Thursday
3,190794,2015-02-18,Desktop,Female,1,1,0,0,February,Wednesday
4,537909,2015-01-15,Desktop,Male,1,0,0,0,January,Thursday


3. Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).

In [269]:
# ТОП месяц по продажам
fg = df.groupby('month')[['payment_confirmation']].sum().reset_index()
fg.sort_values(['payment_confirmation'], ascending=[False]).head(1)

,month,payment_confirmation
2,January,189


In [270]:
# ТОП неделя по продажам
fg = df.groupby('day_of_week')[['payment_confirmation']].sum().reset_index()
fg.sort_values(['payment_confirmation'], ascending=[False]).head(1)

,day_of_week,payment_confirmation
1,Monday,77


4. Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с помощью одного из методов построения сводных таблиц (groupby, pivot_table или crosstab).

In [271]:
# покупатели какого пола и в какой день недели подтвердили оплату на странице "payment_confirmation"
pd.pivot_table(df, index = 'sex', columns = 'day_of_week', values = 'payment_confirmation', aggfunc = 'sum', margins=True)


day_of_week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,All
sex,,,,,,,,
Female,26,43,36,35,31,40,30,241
Male,28,34,34,27,35,29,24,211
All,54,77,70,62,66,69,54,452
